# Wait diagnostics

This recipe shows how to recive an information about all waits of SQL Server, then one transaction will be created and waits will be explored.

## Wait statistics

Following query shows how to observe waits since last restart of SQL Server. We'll never know what statement caused waits, but we can estimate if certain wait is an issue or not. Don't be confised by so many wait types, stay concentrated to waits related to common SQL Server workload.

In [ ]:
select * 
    , wait_time_ms * 1. / waiting_tasks_count as avg_wait_time_ms
from sys.dm_os_wait_stats
where waiting_tasks_count > 0
order by avg_wait_time_ms desc

## Blocks

We will create two transactions, one will block the other. We need to execute following statements in SSMS:

Transaction A: USE Demo go BEGIN TRAN SELECT count(\*) FROM SalesOrders (tablockx)

Transaction B: USE Demo go BEGIN TRAN SELECT count(\*) FROM SalesOrders (tablockx)

In [ ]:
select * from sys.dm_tran_locks where request_session_id in (71, 72)    -- replace the numbers with actual ones from SSMS

Previous statement shows locks, both wanted and aquired. Following statement shows waiting session and also what the session sent to SQL Server.

In [ ]:
select * 
    , s.text
from sys.dm_os_waiting_tasks as t
    join sys.dm_exec_requests as r on t.session_id = r.session_id
    cross apply sys.dm_exec_sql_text(r.sql_handle) as s
where t.session_id = 72 -- replace the number of session with actually waiting session

Let's kill both sessions now. Then we can take a look to the wait\_stats again

In [ ]:
kill 71
kill 72
GO

select * 
    , wait_time_ms * 1. / waiting_tasks_count as avg_wait_time_ms
from sys.dm_os_wait_stats
where waiting_tasks_count > 0
order by avg_wait_time_ms desc
